# Task 2 - False Color Image 

Date: 2024-11-23

Author: Jordan Sibley 

[Github Repository](https://github.com/jordancsibley/eds220-hwk4)

## Data

The data used to create the false color image of the Thomas fire is Landsat data and fire perimeter data. The Landsat Collection 2 Level-2 data is a simplifed collection of bands collected from the an atmosphereic satellite. The fire perimeter data is a subset of a CAL FIRE geodatabase that contains only the geometries for the Thomas Fire. 

In [3]:
# Import libraries 
import pandas as pd 
import geopandas as gpd 
import os 
import rioxarray as rioxr
import xarray as xr

# Set environment variable
os.environ['PROJ_LIB'] = '/opt/anaconda3/share/proj'

ASK IF I SHOULD BE PUTTING LANDSAT DATA IN THE DATA FOLDER OR NOT

In [4]:
# File path to data on workbench 
fp = os.path.join("/courses/EDS220/data/hwk4_landsat_data/landsat8-2018-01-26-sb-simplified.nc")
landsat = rioxr.open_rasterio(fp)